In [ ]:
 !pip install -q opencv - python

ERROR: Invalid requirement: '-'


In [ ]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from moviepy import editor
import moviepy

In [ ]:
def process_video(test_video,output_video):
  input_video = editor.videofileClip(text_video,audio=false)
  processed = input_video.f1_image(frame_processor)
  processed.write_videofile(output_video,audio=false)

In [ ]:
def frame_processor(image):
  grayscale = cv2.cvtcolor(image,cv2.COLOR_BGR2GRAY)
  kernel_size = 5
  blur = cv2.GaussianBlur(grayscale, (kernel_size, kernel_size), 0)
  low_t = 50
  high_t = 150
  edges = cv2.Canny(blur,low_t,high_t)
  region = region_selection(edges)
  hough = hough_transform(region)
  result = draw_lane_lines(image,lane_lines(image,hough))
  return result

In [ ]:
def region_selection(image):
  mask = np.zeros_like(image)
  if len(image.shape) > 2:
    channel_count = image.shape[2]
    ignore_mask_color = (255,) *channel_count
  else:
    ignore_mask_color = 255
  rows, cols = image.shape[:2]
  bottom_left = [cols * 0.1, rows * 0.95]
  top_left = [cols * 0.4,rows * 0.6]
  bottom_right = [cols * 0.9, rows * 0.95]
  top_right = [cols * 0.6,rows * 0.6]
  vertices = np.array([[bottom_left,top_left,top_right,bottom_right]], dtype=np.int32)
  cv2.fillpaly(mask,vertices,ignore_mask_color)
  masked_image = cv2.bitwise_and(image,mask)
  return masked_image

In [ ]:
def hough_transform(image):
  rha = 1
  theta = np.pi/180
  threshold = 20
  minlinelength = 20
  maxlineGap = 580
  return cv2.HoughLinesP(image,rho = rho,theta = theta, threshold = threshold,minLineLength = minLineLength, maxLineGap = maxLineGap)

In [ ]:
def average_slope_intercept(lines):
  left_lines = []
  left_weigths = []
  rigth_lines = []
  rigth_weigths = []

  for line in lines:
    for x1,y1,x2,y2 in line:
      if x1 == x2:
        continue
      slope = (y2 - y1) / (x2 - x1)
      intercept = y1 - (slope * x1)
      length = np.sqrt(((y2 - y1) **2) + ((x2 - x1) **2))
      if slope < 0:
        left_lines.append((slope,intercept))
        left_weigths.append((length))
      else:
        right_lines.append((slope,intercept))
        right_weigths.append((length))

  left_lane = np.dot(left_weigths, left_lines) / np.sum(left_weights) if len(left_weigths) > 0 else none
  rigth_lane = np.dot(right_weigths, rigth_lines) / np.sum(rigth_weights) if len(rigth_weigths) > 0 else none
  return left_lane, right_lane

def pixel_points(y1,y2,line):

  if line is none:
    return none
  slope, intercept = line
  x1 = int ((y1 - intercept)/slope)
  x2 = int ((y2 - intercept)/slope)
  y1 = int(y1)
  y2 = int(y2)
  return ((x1, y1), (x2, y2))

def lane_lines(image,lines):

  left_lane,right_lane = average_slope_intercept(lines)
  y1 = image.shape[0]
  y2 = y1 * 0.6
  left_line = pixel_points(y1,y2,left_lane)
  right_line = pixel_points(y1,y2, right_lane)
  return left_line, right_line

def draw_lane_lines(image,lines,color=[255,0,0], thickness=12):

  line_image = np.zeros_like(image)
  for line in lines:
    if line is not none:
      cv2.line(line_image,*line,color,thickness)
  return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)


